In [ ]:
import os

!mvn clean package

[INFO] Scanning for projects...
[INFO] 
[INFO] --------------< org.sbpo2025.challenge:ChallengeSBPO2025 >--------------
[INFO] Building ChallengeSBPO2025 1.0
[INFO]   from pom.xml
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- clean:3.2.0:clean (default-clean) @ ChallengeSBPO2025 ---
[INFO] Deleting c:\Users\marzu\OneDrive - Universidad de Montevideo (1)\Documentos\MARCO\fing\algoritmos evolutivos\AlgoritmosEvolutivos-challenge-sbpo-2025\target
[INFO] 
[INFO] --- resources:3.3.1:resources (default-resources) @ ChallengeSBPO2025 ---
[INFO] skip non existing resourceDirectory c:\Users\marzu\OneDrive - Universidad de Montevideo (1)\Documentos\MARCO\fing\algoritmos evolutivos\AlgoritmosEvolutivos-challenge-sbpo-2025\src\main\resources
[INFO] 
[INFO] --- compiler:3.13.0:compile (default-compile) @ ChallengeSBPO2025 ---
[INFO] Recompiling the module because of changed source code.
[INFO] Compiling 10 source files with javac [debug target 17

Ejecutar con algoritmo greedy

In [24]:
dataset = "x"

for file in os.listdir(f"datasets/{dataset}"):
    
    print(f"Processing instance: {file}")
    !java -jar target/ChallengeSBPO2025-1.0.jar datasets/{dataset}/{file} greedy
    print("\n")


Processing instance: instance_0001.txt
Processing instance: instance_0001.txt

>> solveGreedySelection
No feasible solution found.


Processing instance: instance_0002.txt


Solution not found


Processing instance: instance_0002.txt

>> solveGreedySelection

Best solution found with value 126.25
Final remaining time: 599 seconds
Total capacity = 10464
Total demand = 505
Total capacity left = 9959 = 95.17392966360856%
Selected aisles = 4
Selected orders = 432
Output written to output/greedy/x/instance_0002.txt


Processing instance: instance_0003.txt
Processing instance: instance_0003.txt

>> solveGreedySelection

Best solution found with value 312.9166666666667
Final remaining time: 599 seconds
Total capacity = 40960
Total demand = 3755
Total capacity left = 37205 = 90.83251953125%
Selected aisles = 12
Selected orders = 2859
Output written to output/greedy/x/instance_0003.txt


Processing instance: instance_0004.txt
Processing instance: instance_0004.txt

>> solveGreedySelection
No feasible solution found.


Processing instance: instance_0005.txt


Solution not found


Processing instance: instance_0005.txt

>> solveGreedySelection

Best solution found with value 324.5
Final remaining time: 599 seconds
Total capacity = 23555
Total demand = 2596
Total capacity left = 20959 = 88.97898535342814%
Selected aisles = 8
Selected orders = 1987
Output written to output/greedy/x/instance_0005.txt


Processing instance: instance_0006.txt
Processing instance: instance_0006.txt

>> solveGreedySelection
No feasible solution found.


Processing instance: instance_0007.txt


Solution not found


Processing instance: instance_0007.txt

>> solveGreedySelection

Best solution found with value 60.5531914893617
Final remaining time: 599 seconds
Total capacity = 20043
Total demand = 2846
Total capacity left = 17197 = 85.80052886294467%
Selected aisles = 47
Selected orders = 2424
Output written to output/greedy/x/instance_0007.txt


Processing instance: instance_0008.txt
Processing instance: instance_0008.txt

>> solveGreedySelection

Best solution found with value 377.0
Final remaining time: 599 seconds
Total capacity = 21950
Total demand = 4147
Total capacity left = 17803 = 81.10706150341686%
Selected aisles = 11
Selected orders = 3681
Output written to output/greedy/x/instance_0008.txt


Processing instance: instance_0009.txt
Processing instance: instance_0009.txt

Processing instance: instance_0010.txt


>> solveGreedySelection

Best solution found with value 777.5
Final remaining time: 599 seconds
Total capacity = 25137
Total demand = 6220
Total capacity left = 18917 = 75.255599

Solution not found


Verificación de factibilidad y valores objetivo

In [29]:
algorithm = "greedy"

for dataset in ["a", "b", "x"]:
    for file in os.listdir(f"output/{algorithm}/{dataset}"):
        !python checker.py datasets/{dataset}/{file} output/{algorithm}/{dataset}/{file}
    

Is solution feasible: True
Objective function value: 2.0
Is solution feasible: True
Objective function value: 307.0
Is solution feasible: True
Objective function value: 196.42857142857142
Is solution feasible: True
Objective function value: 65.66666666666667
Is solution feasible: True
Objective function value: 102.84210526315789
Is solution feasible: True
Objective function value: 101.71428571428571
Is solution feasible: True
Objective function value: 75.5
Is solution feasible: True
Objective function value: 15.875
Is solution feasible: True
Objective function value: 85.125
Is solution feasible: True
Objective function value: 202.0
Is solution feasible: True
Objective function value: 327.3636363636364
Is solution feasible: True
Objective function value: 204.46153846153845
Is solution feasible: True
Objective function value: 52.0
Is solution feasible: True
Objective function value: 175.30769230769232
Is solution feasible: True
Objective function value: 43.567567567567565
Is solution fea